In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import time, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from datetime import datetime

import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam, Adagrad

In [ ]:
import PGNN_source as pg
import NNNN_source as nn

In [ ]:
def compute_test_rmse(y_test, mu_test, offset_test = None):
    if offset_test is None:
        return np.sqrt(np.mean((y_test - mu_test)**2))
    else:
        return np.sqrt(np.mean(((y_test - mu_test)*offset_test)**2))
def compute_test_rmsr(y_test, mu_test, offset_test = None):
    residual = ((y_test - mu_test)**2)/mu_test
    if offset_test is not None:
        residual = residual * offset_test
    residual = residual[residual>0]
    if np.sum(residual<=0) >0: print(str(np.sum(residual<=0))+' were excluded for rmsr')
    return np.sqrt(np.mean(residual))

In [ ]:
def make_mean_model(nodes, layers):    
    input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
    input_Z = Input(shape=(np.shape(Z_train)[1],), dtype='float32')
    m  = Dense(nodes, activation=activation)(input_X)
    for i in range(layers-1):
        m  = Dense(nodes, activation=activation)(m)         
    xb = Dense(1, activation='linear')(m)
    zv = Dense(1, activation='linear', use_bias=False)(input_Z)    
    mean_model = Model(inputs=[input_X, input_Z], outputs=[xb, zv])
    return mean_model

def make_mean_model_Nw(nodes, layers):    
    input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
    m  = Dense(nodes, activation=activation)(input_X)
    for i in range(layers-1):
        m  = Dense(nodes, activation=activation)(m)          
    xb = Dense(1, activation='linear')(m)    
    mean_model = Model(inputs=[input_X], outputs=[xb])
    return mean_model

def make_mean_model_Nf(nodes, layers):    
    input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
    input_Z = Input(shape=(np.shape(Z_train)[1],), dtype='float32')
    m  = Dense(nodes, activation='sigmoid')(input_X)
    for i in range(layers-1):
        m  = Dense(nodes, activation=activation)(m)         
    xb = Dense(1, activation='linear')(m)
    zv = Dense(1, activation='linear', use_bias=False)(input_Z)        
    mean_model = Model(inputs=[input_X, input_Z], outputs=[xb+zv])
    return mean_model

def make_mean_model_Pw(nodes, layers):    
    input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
    m  = Dense(nodes, activation='sigmoid')(input_X)
    for i in range(layers-1):
        m  = Dense(nodes, activation=activation)(m)             
    expxb = Dense(1, activation='exponential')(m)        
    mean_model = Model(inputs=[input_X], outputs=[expxb])
    return mean_model

def make_mean_model_Pf(nodes, layers):    
    input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
    input_Z = Input(shape=(np.shape(Z_train)[1],), dtype='float32')
    m  = Dense(nodes, activation='sigmoid')(input_X)
    for i in range(layers-1):
        m  = Dense(nodes, activation=activation)(m)             
    expxb = Dense(1, activation='exponential')(m)
    expzv = Dense(1, activation='exponential', use_bias=False)(input_Z)        
    mean_model = Model(inputs=[input_X, input_Z], outputs=[expxb*expzv])
    return mean_model

def make_mean_model_HL():    
    input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
    input_Z = Input(shape=(np.shape(Z_train)[1],), dtype='float32')    
    xb = Dense(1, activation='linear')(input_X)
    zv = Dense(1, activation='linear', use_bias=False)(input_Z)
    mean_model = Model(inputs=[input_X, input_Z], outputs=[xb, zv])
    return mean_model

In [ ]:
def analyze_dataset(
    model_names, X_train, Z_train, y_train, X_valid, Z_valid, y_valid, X_test, Z_test, y_test, nodes, layers, activation, 
    phi_init = 1., lam_init = 1., patience = 10, pretrain = 100, max_epochs = 2000, lr = 0.001, offset_test = None
):

    optimizer = Adam(learning_rate=lr)
    callbacks = [EarlyStopping(monitor='val_loss', patience=patience)]
    
    N_train = len(y_train)
    batch_size, batch_ratio = N_train, 1.
    
    pg.seed_everything()
    train_batch = tf.data.Dataset.from_tensor_slices((X_train, Z_train, y_train)).shuffle(N_train).batch(N_train)
    
    res_time, res_rmse, res_rmsr, res_mu = {}, {}, {}, {}

    # N-NN
    if 'Nw' in model_names:
        K.clear_session() 
        pg.seed_everything()    
        M = make_mean_model_Nw(nodes, layers)
        M.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
        start_time = time.time()
        M_history = M.fit([X_train], y_train, epochs=max_epochs, batch_size=batch_size, verbose=0, 
            callbacks=callbacks, validation_data=([X_valid], y_valid))
        mu_test = np.float32(M([X_test])).T
        res_time['Nw'] = time.time() - start_time
        res_rmse['Nw'] = compute_test_rmse(y_test, mu_test, offset_test)
        res_rmsr['Nw'] = compute_test_rmsr(y_test, mu_test, offset_test)
        res_mu['Nw'] = mu_test

    # NF-NN
    if 'Nf' in model_names:
        K.clear_session() 
        pg.seed_everything()
        M = make_mean_model_Nf(nodes, layers)
        M.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
        start_time = time.time()
        M_history = M.fit([X_train, Z_train], y_train, epochs=max_epochs, batch_size=batch_size, verbose=0, 
            callbacks=callbacks, validation_data=([X_valid, Z_valid], y_valid))
        mu_test = np.float32(M([X_test, Z_test])).T
        res_time['Nf'] = time.time() - start_time
        res_rmse['Nf'] = compute_test_rmse(y_test, mu_test, offset_test)
        res_rmsr['Nf'] = compute_test_rmsr(y_test, mu_test, offset_test)
        res_mu['Nf'] = mu_test

    # NN-NN
    if 'NN' in model_names:
        K.clear_session() 
        pg.seed_everything()
        M = make_mean_model(nodes, layers)
        start_time = time.time()
        res = nn.train_model(M, train_batch, [X_train, Z_train, y_train], [X_valid, Z_valid, y_valid],
             nn.nn_hlik_loss, optimizer, phi_init, lam_init, batch_ratio, patience, pretrain, max_epochs)
        mu_test = np.sum(M([X_test, Z_test]), axis=0).T
        res_time['NN'] = time.time() - start_time
        res_rmse['NN'] = compute_test_rmse(y_test, mu_test, offset_test)
        res_rmsr['NN'] = compute_test_rmsr(y_test, mu_test, offset_test)
        res_mu['NN'] = mu_test    
    
    # P-NN
    if 'Pw' in model_names:
        K.clear_session() 
        pg.seed_everything()
        M = make_mean_model_Pw(nodes, layers)
        M.compile(optimizer=optimizer, loss=tf.keras.losses.Poisson())
        start_time = time.time()
        M_history = M.fit([X_train], y_train, epochs=max_epochs, batch_size=batch_size, verbose=0, 
            callbacks=callbacks, validation_data=([X_valid], y_valid))
        mu_test = np.float32(M([X_test])).T
        res_time['Pw'] = time.time() - start_time
        res_rmse['Pw'] = compute_test_rmse(y_test, mu_test, offset_test)
        res_rmsr['Pw'] = compute_test_rmsr(y_test, mu_test, offset_test)
        res_mu['Pw'] = mu_test

    # PF-NN
    if 'Pf' in model_names:
        K.clear_session() 
        pg.seed_everything()
        M = make_mean_model_Pf(nodes, layers)
        M.compile(optimizer=optimizer, loss=tf.keras.losses.Poisson())
        start_time = time.time()
        M_history = M.fit([X_train, Z_train], y_train, epochs=max_epochs, batch_size=batch_size, verbose=0, 
            callbacks=callbacks, validation_data=([X_valid, Z_valid], y_valid))
        mu_test = np.float32(M([X_test, Z_test])).T
        res_time['Pf'] = time.time() - start_time
        res_rmse['Pf'] = compute_test_rmse(y_test, mu_test, offset_test)
        res_rmsr['Pf'] = compute_test_rmsr(y_test, mu_test, offset_test)
        res_mu['Pf'] = mu_test

    # PG-NN
    if 'PG' in model_names:
        K.clear_session() 
        pg.seed_everything()
        M = make_mean_model(nodes, layers)
        start_time = time.time()
        res = pg.train_model(M, train_batch, [X_train, Z_train, y_train], [X_valid, Z_valid, y_valid],
             pg.pg_hlik_loss, optimizer, lam_init, batch_ratio, patience, pretrain, max_epochs)
        mu_test = np.exp(np.sum(M([X_test, Z_test]), axis=0).T)
        res_time['PG'] = time.time() - start_time
        res_rmse['PG'] = compute_test_rmse(y_test, mu_test, offset_test)
        res_rmsr['PG'] = compute_test_rmsr(y_test, mu_test, offset_test)
        res_mu['PG'] = mu_test

    # PG-GLM
    if 'HL' in model_names:
        K.clear_session() 
        pg.seed_everything()
        M = make_mean_model_HL()
        start_time = time.time()
        res = pg.train_model(M, train_batch, [X_train, Z_train, y_train], [X_valid, Z_valid, y_valid],
             pg.pg_hlik_loss, optimizer, lam_init, batch_ratio, patience, pretrain, max_epochs)
        mu_test = np.exp(np.sum(M([X_test, Z_test]), axis=0).T)
        res_time['HL'] = time.time() - start_time
        res_rmse['HL'] = compute_test_rmse(y_test, mu_test, offset_test)
        res_rmsr['HL'] = compute_test_rmsr(y_test, mu_test, offset_test)
        res_mu['HL'] = mu_test
        
    return res_time, res_rmse, res_rmsr

In [ ]:
dir_name = os.getcwd()+'/data/'

activation = 'sigmoid'
nodes, layers = 10, 1
lr = 0.01
phi_init, lam_init = 0.5, 0.5
patience, pretrain, max_epochs = 10, 100, 2000

# Epilepsy data

In [ ]:
data = pd.read_csv(dir_name+'epilepsy.csv')
N, n_sub, n_num = 236, 59, 4

split_num = np.zeros(N)
pg.seed_everything()
for i in range(n_sub):
    split_num[i * n_num + np.random.choice((n_num-1), 1)] = 1
split_num = split_num + 2.*(data['time']==4)
data['split_num'] = split_num

data_trvl = data[data['time']!=4]
data_test = data[data['time']==4]
data_train = data_trvl[data_trvl['split_num']==0]
data_valid = data_trvl[data_trvl['split_num']==1]

covariate_names = ['time', 'drug', 'base', 'age']
subset_names = ['_train', '_valid', '_test', '_trvl']
for subset in subset_names:
    exec('temp_data = data'+subset)
    exec('N'+subset+'= np.shape(temp_data)[0]')
    exec('X'+subset+'= np.array(temp_data[covariate_names], dtype=np.float32)')
    exec('y'+subset+'= np.array(temp_data["y"], dtype=np.float32)')        
    exec('z'+subset+'= np.array(temp_data["id"].astype("int32"))')
    exec('Z'+subset+'= np.eye(n_sub)[z'+subset+'].astype("float32")')

In [ ]:
model_names = ['Nw','Nf','NN','Pw','Pf','PG']
times, rmses, rmsrs = analyze_dataset(
    model_names, X_trvl, Z_trvl, y_trvl, X_valid, Z_valid, y_valid, X_test, Z_test, y_test,
    nodes, layers, activation, phi_init, lam_init, patience, pretrain, max_epochs)
pd.DataFrame([np.round(list(rmses.values()),3),np.round(list(rmsrs.values()),3)], columns = model_names, index=['RMSE', 'RMSR'])

# CD4 data

In [ ]:
data = pd.read_csv(dir_name+'cd4.csv')

data['y'] = data['cd4']
data['zA225z'] = 1*(data['treatment'] == 'zA225z')
data['zA400d'] = 1*(data['treatment'] == 'zA400d')
data['zX400d'] = 1*(data['treatment'] == 'zX400d')

N = np.shape(data)[0] # = 4612
n_sub = np.shape(np.unique(data['id']))[0] # = 1038
q = np.zeros(n_sub, dtype=np.int32)
for i in range(n_sub):
    q[i] = int(np.sum(data['id']==i))

split_num = np.zeros(N)
pg.seed_everything()
count = 0
for i in range(n_sub):
    split_num[count + np.random.choice((q[i]-1), 1)] = 1
    count += q[i]
split_num = split_num + 2.*(data['last_visit']==1)
data['split_num'] = split_num

data_trvl = data[data['last_visit']!=1]
data_test = data[data['last_visit']==1]
data_train = data_trvl[data_trvl['split_num']==0]
data_valid = data_trvl[data_trvl['split_num']==1]

covariate_names = ['age', 'gender','week','zA225z', 'zA400d', 'zX400d']
subset_names = ['_train', '_valid', '_test', '_trvl']
for subset in subset_names:
    exec('temp_data = data'+subset)
    exec('N'+subset+'= np.shape(temp_data)[0]')
    exec('X'+subset+'= np.array(temp_data[covariate_names], dtype=np.float32)')
    exec('y'+subset+'= np.array(temp_data["y"], dtype=np.float32)')        
    exec('z'+subset+'= np.array(temp_data["id"].astype("int32"))')
    exec('Z'+subset+'= np.eye(n_sub)[z'+subset+'].astype("float32")')

In [ ]:
model_names = ['Nw','Nf','NN','Pw','Pf','PG']
times, rmses, rmsrs = analyze_dataset(
    model_names, X_trvl, Z_trvl, y_trvl, X_valid, Z_valid, y_valid, X_test, Z_test, y_test,
    nodes, layers, activation, phi_init, lam_init, patience, pretrain, max_epochs)
pd.DataFrame([np.round(list(rmses.values()),3),np.round(list(rmsrs.values()),3)], columns = model_names, index=['RMSE', 'RMSR'])

# Bolus data

In [ ]:
data = pd.read_csv(dir_name+'bolus.csv')
N, n_sub, n_num = 780, 65, 12

data['1mg'] = data['group']=='1mg'
data['2mg'] = data['group']=='2mg'
split_num = np.zeros(N)
pg.seed_everything()
for i in range(n_sub):
    split_num[i * n_num + np.random.choice((n_num-1), 1)] = 1
split_num = split_num + 2.*(data['time']==12)
data['split_num'] = split_num

data_trvl = data[data['time']!=12]
data_test = data[data['time']==12]
data_train = data_trvl[data_trvl['split_num']==0]
data_valid = data_trvl[data_trvl['split_num']==1]

covariate_names = ['time', '2mg', '1mg']
subset_names = ['_train', '_valid', '_test', '_trvl']
for subset in subset_names:
    exec('temp_data = data'+subset)
    exec('N'+subset+'= np.shape(temp_data)[0]')
    exec('X'+subset+'= np.array(temp_data[covariate_names], dtype=np.float32)')
    exec('y'+subset+'= np.array(temp_data["y"], dtype=np.float32)')        
    exec('z'+subset+'= np.array(temp_data["id"].astype("int32"))')
    exec('Z'+subset+'= np.eye(n_sub)[z'+subset+'].astype("float32")')

In [ ]:
model_names = ['Nw','Nf','NN','Pw','Pf','PG','HL']
times, rmses, rmsrs = analyze_dataset(
    model_names, X_train, Z_train, y_train, X_valid, Z_valid, y_valid, X_test, Z_test, y_test,
    nodes, layers, activation, phi_init, lam_init, patience, pretrain, max_epochs, lr)
pd.DataFrame([np.round(list(rmses.values()),3),np.round(list(rmsrs.values()),3)], columns = model_names, index=['RMSE', 'RMSR'])

# Owls data

In [ ]:
data = pd.read_csv(dir_name+'owls.csv')

data['FT'] = 1*(data['FoodTreatment']=='Satiated')
data['SP'] = 1*(data['SexParent']=='Male')
data['y']  = data['SiblingNegotiation']/data['BroodSize']

N = np.shape(data)[0]
n_sub = np.shape(np.unique(data['id']))[0]
split_num = np.zeros(N)
q = np.zeros(n_sub, dtype=np.int32)
for i in range(n_sub): q[i] = int(np.sum(data['id']==i))    
    
pg.seed_everything()
for i in range(n_sub):
    count = 0; valid_num, test_num = np.random.choice(q[i], 2, replace=False)
    for j in range(N):
        if data['id'][j]==i:
            if count == valid_num: split_num[j] = 1
            elif count == test_num: split_num[j] = 2
            count += 1
data['split'] = split_num

data_valid = data[data['split']==1]
data_train = data[data['split']==0]
data_test = data[data['split']==2]
data_trvl = data[data['split']!=2]

covariate_names = ['ArrivalTime', 'FT', 'SP']
subset_names = ['_train', '_valid', '_test', '_trvl']
for subset in subset_names:
    exec('temp_data = data'+subset)
    exec('N'+subset+'= np.shape(temp_data)[0]')
    exec('X'+subset+'= np.array(temp_data[covariate_names], dtype=np.float32)')
    exec('y'+subset+'= np.array(temp_data["y"], dtype=np.float32)')        
    exec('z'+subset+'= np.array(temp_data["id"].astype("int32"))')
    exec('Z'+subset+'= np.eye(n_sub)[z'+subset+'].astype("float32")')

offset_test = np.float32(data_test['BroodSize'])

In [ ]:
model_names = ['Nw','Nf','NN','Pw','Pf','PG','HL']
times, rmses, rmsrs = analyze_dataset(
    model_names, X_train, Z_train, y_train, X_valid, Z_valid, y_valid, X_test, Z_test, y_test,
    nodes, layers, activation, phi_init, lam_init, patience, pretrain, max_epochs, lr, offset_test)
pd.DataFrame([np.round(list(rmses.values()),3),np.round(list(rmsrs.values()),3)], columns = model_names, index=['RMSE', 'RMSR'])

# Fruits data

In [ ]:
data = pd.read_csv(dir_name+'fruits.csv')

data['amd'] = 1*(data['amd']=='clipped')
data['rack'] = 1*(data['rack']==2) 
data['normal'] = 1*(data['status']=='Normal') 
data['trans'] = 1*(data['status'] =='Transplant') 
data['petri'] = 1*(data['status'] =='Petri.Plate') 
data['y']  = data['total.fruits']

N = np.shape(data)[0]
n_sub = np.shape(np.unique(data['id']))[0]

split_num = np.zeros(N)
q = np.zeros(n_sub, dtype=np.int32)
for i in range(n_sub): q[i] = int(np.sum(data['id']==i))
pg.seed_everything()
for i in range(n_sub):
    count = 0; valid_num, test_num = np.random.choice(q[i], 2, replace=False)
    for j in range(N):
        if data['id'][j]==i:
            if count == valid_num: split_num[j] = 1
            elif count == test_num: split_num[j] = 2
            count += 1
data['split'] = split_num

data_valid = data[data['split']==1]
data_train = data[data['split']==0]
data_test = data[data['split']==2]
data_trvl = data[data['split']!=2]

covariate_names = ['nutrient', 'amd', 'rack', 'normal','trans', 'petri']
subset_names = ['_train', '_valid', '_test', '_trvl']
for subset in subset_names:
    exec('temp_data = data'+subset)
    exec('N'+subset+'= np.shape(temp_data)[0]')
    exec('X'+subset+'= np.array(temp_data[covariate_names], dtype=np.float32)')
    exec('y'+subset+'= np.array(temp_data["y"], dtype=np.float32)')        
    exec('z'+subset+'= np.array(temp_data["id"].astype("int32"))')
    exec('Z'+subset+'= np.eye(n_sub)[z'+subset+'].astype("float32")')

In [ ]:
model_names = ['Nw','Nf','NN','Pw','Pf','PG','HL']
times, rmses, rmsrs = analyze_dataset(
    model_names, X_train, Z_train, y_train, X_valid, Z_valid, y_valid, X_test, Z_test, y_test,
    nodes, layers, activation, phi_init, lam_init, patience, pretrain, max_epochs, lr)
pd.DataFrame([np.round(list(rmses.values()),3),np.round(list(rmsrs.values()),3)], columns = model_names, index=['RMSE', 'RMSR'])